In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import numpy as np

In [12]:
def eval(model, test_images,  test_labels):
  test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
  print(test_acc)

  predictions = model.predict(test_images)
  predicted_labels = tf.argmax(predictions,axis=1)

  confusion = confusion_matrix(test_labels, predicted_labels)
  precision = precision_score(test_labels, predicted_labels, average='weighted')
  recall = recall_score(test_labels, predicted_labels, average='weighted')
  f1 = f1_score(test_labels, predicted_labels, average='weighted')

  print("Confusion Matrix:")
  print(confusion)
  print(f"Precision: {precision:.4f}")
  print(f"Recall: {recall:.4f}")
  print(f"F1 Score:{f1:.4f}")


In [13]:
(xtrain,ytrain),(xtest,ytest)= keras.datasets.mnist.load_data()
xtrain=np.dstack([xtrain] * 3)
xtest=np.dstack([xtest]*3)

In [14]:
xtrain = xtrain.reshape(-1, 28,28,3)
xtest= xtest.reshape (-1,28,28,3)
xtrain.shape

(60000, 28, 28, 3)

In [15]:
from keras.preprocessing.image import img_to_array, array_to_img

xtrain = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in xtrain])
xtest = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in xtest])

In [16]:
from tensorflow.keras.applications import VGG16
input_layer=layers.Input(shape=(48,48,3))
model_vgg16=VGG16(weights='imagenet',input_tensor=input_layer,include_top=False)

model_vgg16.summary()


58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                              

In [17]:
last_layer=model_vgg16.output
flatten=layers.Flatten()(last_layer)
dense1=layers.Dense(4096,activation='relu')(flatten)
dense1=layers.Dense(4096,activation='relu')(flatten)
output_layer=layers.Dense(10,activation='softmax')(flatten)
model=models.Model(inputs=input_layer,outputs=output_layer)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

In [18]:
for layer in model.layers[:-1]:
    layer.trainable=False
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

In [19]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history2 = model.fit(xtrain, ytrain, epochs=5,
                    validation_data=(xtest, ytest))

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1875/1875 [==============================] - 28s 14ms/step - loss: 0.9422 - accuracy: 0.8318 - val_loss: 0.3495 - val_accuracy: 0.9071
Epoch 2/5
1875/1875 [==============================] - 27s 14ms/step - loss: 0.3563 - accuracy: 0.9041 - val_loss: 0.3250 - val_accuracy: 0.9090
Epoch 3/5
1875/1875 [==============================] - 27s 14ms/step - loss: 0.3272 - accuracy: 0.9110 - val_loss: 0.3231 - val_accuracy: 0.9130
Epoch 4/5
1875/1875 [==============================] - 26s 14ms/step - loss: 0.3150 - accuracy: 0.9146 - val_loss: 0.3382 - val_accuracy: 0.9120
Epoch 5/5
1875/1875 [==============================] - 26s 14ms/step - loss: 0.3070 - accuracy: 0.9150 - val_loss: 0.4062 - val_accuracy: 0.8934


In [20]:
eval(model,xtest, ytest)

313/313 - 4s - loss: 0.4062 - accuracy: 0.8934 - 4s/epoch - 11ms/step
0.8934000134468079
313/313 [==============================] - 4s 11ms/step
Confusion Matrix:
[[ 969    0    1    0    0    3    2    1    1    3]
 [   1 1104    2    1    9    5    4    4    1    4]
 [  13    1  875   13    6   66    8    9   14   27]
 [   0    0    8  829    0  154    0    2    6   11]
 [   1    0    7    1  896    8    5    2    2   60]
 [   6    1    5   19    0  841    5    5    6    4]
 [  24    3   12    0   11   20  882    0    3    3]
 [   1    2   16    6   16   32    0  822    1  132]
 [  11    1    9   24   14   77    2    1  734  101]
 [   3    0    2    4    7    7    0    2    2  982]]
Precision: 0.9085
Recall: 0.8934
F1 Score:0.8950


In [21]:
model1 = models.Sequential()

model1.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(48, 48, 3)))
model1.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model1.add(layers.MaxPooling2D((2, 2), strides=(2,2)))

model1.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model1.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model1.add(layers.MaxPooling2D((2, 2), strides=(2,2)))

model1.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model1.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model1.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model1.add(layers.MaxPooling2D((2, 2), strides=(2,2)))

model1.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model1.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model1.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model1.add(layers.MaxPooling2D((2, 2), strides=(2,2)))

model1.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model1.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model1.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model1.add(layers.MaxPooling2D((2, 2), strides=(2,2)))

model1.add(layers.Flatten())
model1.add(layers.Dense(4096, activation='relu'))
model1.add(layers.Dense(4096, activation='relu'))
model1.add(layers.Dense(10, activation='softmax'))

model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 48, 48, 64)        1792      
                                                                 
 conv2d_14 (Conv2D)          (None, 48, 48, 64)        36928     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 24, 24, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 24, 24, 128)       73856     
                                                                 
 conv2d_16 (Conv2D)          (None, 24, 24, 128)       147584    
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 12, 12, 128)       0         
 g2D)                                                 

In [22]:
model1.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history2 = model1.fit(xtrain, ytrain,batch_size=128, epochs=5,
                    validation_data=(xtest, ytest))

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


469/469 [==============================] - 54s 108ms/step - loss: 2.8181 - accuracy: 0.1167 - val_loss: 2.3019 - val_accuracy: 0.1135
Epoch 2/5
469/469 [==============================] - 50s 107ms/step - loss: 2.3022 - accuracy: 0.1155 - val_loss: 2.3032 - val_accuracy: 0.0958
Epoch 3/5
469/469 [==============================] - 50s 107ms/step - loss: 1.2616 - accuracy: 0.5278 - val_loss: 0.1829 - val_accuracy: 0.9533
Epoch 4/5
469/469 [==============================] - 51s 109ms/step - loss: 0.1401 - accuracy: 0.9621 - val_loss: 0.1235 - val_accuracy: 0.9660
Epoch 5/5
469/469 [==============================] - 51s 109ms/step - loss: 0.0868 - accuracy: 0.9765 - val_loss: 0.0879 - val_accuracy: 0.9730


In [23]:
eval(model1,xtest, ytest)

313/313 - 4s - loss: 0.0879 - accuracy: 0.9730 - 4s/epoch - 12ms/step
0.9729999899864197
313/313 [==============================] - 4s 11ms/step
Confusion Matrix:
[[ 979    0    0    0    0    0    0    1    0    0]
 [   0 1132    1    1    0    0    0    0    1    0]
 [  23   16  988    0    0    0    0    2    3    0]
 [   0    0    3  982    0    8    0    0    9    8]
 [   6    2    0    0  953    0    6    0    4   11]
 [   0    9    0    4    2  863    3    0    9    2]
 [  37   10    2    0    1    0  906    0    2    0]
 [   1    9   10    1    3    0    0  995    2    7]
 [   6    0    1    0    0    1    4    0  961    1]
 [   0    0    0    1   19    0    0    8   10  971]]
Precision: 0.9736
Recall: 0.9730
F1 Score:0.9730
